In [1]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd
import uuid
import json

# Replace these with your actual PostgreSQL credentials
user = "hrm"
password = "hrm"
host = "35.223.205.197"  # IP address or hostname
port = "5432"  # Default PostgreSQL port
database = "attendance_tracking"

# user= "postgres"
# password= "123456"
# host= "localhost"
# port= "5432"
# database= "attendance_tracking"

# Database connection URI for PostgreSQL
pg_database_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'
pg_engine = create_engine(pg_database_uri)

# SQLite database URI
sqlite_database_uri = 'sqlite:///local_attendance_tracking.db'
sqlite_engine = create_engine(sqlite_database_uri)

# Create MetaData instance for PostgreSQL
pg_metadata = MetaData()
pg_metadata.reflect(bind=pg_engine)

# Specify the tables you want to copy
tables_to_copy = ['departments','workshops','user_images', 'user_details', 'tracking_history', 'tracking_history_error']

# Iterate over specified tables and copy schema and data to SQLite
for table_name in tables_to_copy:
    if table_name in pg_metadata.tables:
        print(f"Copying table: {table_name}")
        pg_table = pg_metadata.tables[table_name]

        # Fetch data from PostgreSQL table
        data = pd.read_sql_table(table_name, pg_engine)

        # Check if the DataFrame is not empty before processing
        if not data.empty:
            # Convert UUID columns to strings and handle complex data
            for col in data.columns:
                if data[col].dtype == 'object':
                    # Convert UUIDs to strings
                    if len(data[col]) > 0 and isinstance(data[col].iloc[0], uuid.UUID):
                        data[col] = data[col].astype(str)

                    # Convert dictionaries or JSON-like objects to strings
                    elif isinstance(data[col].iloc[0], dict):
                        data[col] = data[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

        # Write data to SQLite table
        data.to_sql(table_name, sqlite_engine, if_exists='replace', index=False)

print("Database copy complete.")

Copying table: departments
Copying table: workshops
Copying table: user_images
Copying table: user_details
Copying table: tracking_history
Copying table: tracking_history_error
Database copy complete.


In [1]:
#check db
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, select
import psycopg2

# Connect to PostgreSQL database
pg_engine = create_engine('sqlite:///local_attendance_tracking.db')
online_connection = pg_engine.connect()
online_metadata = db.MetaData()

user_details = db.Table('user_details', online_metadata, autoload_with=pg_engine)
user_images = db.Table('user_images', online_metadata, autoload_with=pg_engine)
tracking_history = db.Table('tracking_history', online_metadata, autoload_with=pg_engine)
tracking_history_error = db.Table('tracking_history_error', online_metadata, autoload_with=pg_engine)

NoSuchTableError: user_details

In [6]:
# Truy vấn dữ liệu từ bảng
select_query = db.select(user_details)
result_proxy = online_connection.execute(select_query)
result_set = result_proxy.fetchall()

# Hiển thị kết quả
for row in result_set:
    print(row)

('853e98fa-5ce2-4d65-a641-51ae522739b2', '167787b5-bee7-44f0-a868-0d6d6267a8b0', 'NT12333', 'c552b23e-f334-4efd-9dd7-bb59d93bb85c', '$2a$10$w7Ge4SKkXjL4CbDhjBooO.7t.4fHagKRBmPXvhTsF982J35Ic2Mha', 'Nguyễn Tùng Chi', '2000-08-16', 5, '', '', '', '', '', '', '', '', '', '', 'Nam', '0948845991', '', '11111111', '2012-05-09', '', '', '', '', '', '', '', '', '', '0001-01-01', '0001-01-01', '0001-01-01', '', '', '', '', 1717489269.0, 1717489269, 0, 1, '0001-01-01', 0, None, None, None)
('11d68fc2-3ade-401e-b064-4ee458fe55bb', 'af3c1ab3-156a-4fa4-b3f3-a76a149c9cae', '5564', '91a26ba0-9bbe-4eb6-b972-fef6bf2d6791', '$2a$10$uHTbH4/QnmHv2/yNXH1msOsU5Dw9O9a9voWnYtu9gjZcSh2PzIvSK', 'Vũ Thị Bắc', '2001-03-27', 5, 'Xóm 2 - Tân Thành - Vụ Bản - Nam Định', 'Tân Thành', 'Vụ Bản', 'Nam Định', 'Nam Định', 'Vụ Bản', 'Tân Thành', 'Xóm 2 - Tân Thành - Vụ Bản - Nam Định', 'Việt Nam ', 'Kinh', 'Nữ', '0855566296', '', '036301004801', '2017-09-08', 'Cục cảnh sát đăng ký quản lý cư trú và dữ liệu Quốc gia về dân c

In [7]:
engine = db.create_engine("sqlite:///local_attendance_tracking.db")
connection = engine.connect()
metadata = db.MetaData()
user_details = db.Table('user_details', metadata, autoload_with=engine)
user_images = db.Table('user_images', metadata, autoload_with=engine)
tracking_history = db.Table('tracking_history', metadata, autoload_with=engine)
tracking_history_error = db.Table('tracking_history', metadata, autoload_with=engine)